## Introdução a Machine Learning em uma Competição do Kaggle: Titanic

<img src="https://github.com/mirianfsilva/titanic-kaggle-competition/blob/master/en-us/images/titanic.jpg?raw=true" width="800" style="float:left"/>

## 📌 Tarefa 1: Introdução ao Kaggle
    Criando sua conta no Kaggle e se inscrevendo na competição
---
- Crie sua conta no Kaggle e participe da competição em: [**Titanic: Machine Learning from Disaster**](https://www.kaggle.com/c/titanic).
- Leia a descrição e as informações sobre a competição. 
- Faça download do dataset que será usado nesse projeto, que se encontra na página da competição. 

Vamos das inicio ao projeto!!! 

## 📌 Tarefa 2: Análise Exploratória dos Dados (Exploratory Data Analysis - EDA)


Importe as bibliotecas que inicialmente serão utilizadas

In [ ]:
#necessary installations
#python +3, pandas, pandas profiling
%matplotlib inline
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) 
import matplotlib.pyplot as plt #plot our graphics and visualize our data
import seaborn as sns #plot beautiful graphics :)
import pandas_profiling as pp #pandas profiling analyse our entire dataset and facilate our work in a EDA
import warnings
warnings.filterwarnings('ignore')

### Importe o conjunto de dados
---

In [ ]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

### Visualização dos Dados
---


In [ ]:
#Pandas Profiling Report

report = pp.ProfileReport(train_data)
display(report)

**Nota sobre as variáveis**

_pclass_: Classe socioeconômica/Status dos passageiros.
- _1st_ = Upper
- _2nd_ = Middle
- _3rd_ = Lower

_age_: Idade

_sibsp_: Tipo familiar
- _Sibling_ = irmão, irmã
- _Spouse_ = marido, esposa

_parch_: O conjunto de dados define os relacionamento familiares da forma:
- _Parent_ = mother, father
- _Child_ = daughter, son, stepdaughter, stepson

**Exporte seu relatório**

In [ ]:
# O comando a seguir irá exportar seu relatório em um arquivo html
report.to_file(output_file="dataframe_titanic_report.html")

## 📌 Tarefa 3: Pré-processamento I
    Analisando Dados Faltantes
---

**Important:** Dados Faltantes/Omissos são dados que literalmente estão em falta no conjunto de dados e podem ser importantes para o resultado da análise. Trabalhar com um dataset com dados faltantes é um problema de grande relevância na área de Análise de Dados e esse problema pode ser originado devido a inúmeras causas e fontes, como falhas no sistema de coleda dos dados, integração com diferentes fontes ao criar o dataset, etc., o ponto é que devemos tomar cuidado para evitar viés no resulta do que buscamos, devido a essa lacuna de dados.

In [ ]:
'''
Descriptive statistics include those that summarize the central tendency, 
dispersion and shape of a dataset’s distribution, excluding NaN values.
'''
train_data.describe(include="all")

Depois de visualizar todo o nosso conjunto de dados, **verifique a descrição acima ou a amostra dos dados em nosso relatório de dataframe**, você verá que existem certos pontos de dados rotulados com um `NaN`. Isso denota valores ausentes. Conjuntos de dados diferentes codificam valores ausentes de maneiras diferentes. Às vezes pode ser `9999`, outras vezes` 0` - porque os dados do mundo real podem ser muito confusos!

**O objetivo aqui é descobrir a melhor forma de processar os dados para que nosso modelo de aprendizado de máquina possa aprender com eles.**

### Pré processamento
---




**Observe os valores numéricos e categóricos separadamente:**

Características numéricas: Idade, tarifa, SibSp, Parch.
Características categóricas: sobrevivido, sexo, embarcado, Pclass.
Características alfanuméricas (mas categóricas): Ingresso, Cabine.

Pela visão geral do relatório, clique na tab "Warnings" para ver mais:

- Ingressos e Cabine são características com alta cardinalidade e muitos valores diferenciados.
- Idade e cabine tem muitos valores ausentes.
- Nome e ID possuem valores únicos.
- SibSp, Parch e Fare tem muitos zeros.

In [ ]:
train_data.head()

**Quantas pessoas sobreviveram?**

In [ ]:
plt.figure(figsize=(20, 2))
sns.countplot(y ="Survived",palette="pastel",data=train_data)
plt.title("Not Survived vs Survived")
plt.show()

--- 
**Feature Nome**

In [ ]:
#Feature: Name and ID. We won't move forward using the name variable.
train_data = train_data.drop(["Name"], axis=1)
test_data = test_data.drop(["Name"], axis=1)

---
**Feature Idade**

In [ ]:
plt.figure(figsize=(20, 5))
sns.boxplot(x="Survived", y="Age", palette="pastel", data=train_data)

In [ ]:
#Feature: Age. We have some missing values in Age, 117 missing, we won't move forward with this.
train_data = train_data.drop(["Age"], axis = 1)
test_data = test_data.drop(["Age"], axis = 1)

---
**Feature Ticket**

In [ ]:
#See plot inside our report. Tickets vs frequency

In [ ]:
#Feature: Ticket. We won't use Ticket.
train_data = train_data.drop(["Ticket"], axis=1)
test_data = test_data.drop(["Ticket"], axis=1)

--- 
**Feature Cabine**

In [ ]:
#See plot inside our report. Cabin vs frequency of distribution

In [ ]:
#Feature: Cabin. Too many missing values, we won't move forward with this.
train_data = train_data.drop(["Cabin"], axis=1)
test_data = test_data.drop(["Cabin"], axis=1)

## 📌 Tarefa 4: Pré-processamento II
    Analisando Dados Faltantes
---
**Feature Embarque**

In [ ]:
sns.countplot(y='Embarked', palette="pastel", data=train_data);

In [ ]:
#We have 2 missing values in the feature embarked. We will Drop these 2 missing values only in the train set
train_data = train_data.dropna(subset=["Embarked"])

'''
Other option, is fill the 2 missing values with the place where the majority of people embarked 
According our histogram plotted in our report, Southampton is the most frequent
replacing the missing values in the Embarked feature with S
'''
#train_data = train_data.fillna({"Embarked": "S"})

---
**Feature Sexo**

In [ ]:
#Sex Feature: map each Sex value to a numerical value
#0 for male and 1 for female
train_data["Sex"] = np.where(train_data["Sex"] == "female", 1, 0)
test_data["Sex"] = np.where(test_data["Sex"] == "female", 1, 0)

# Let's view the distribution of Sex in our dataset
plt.figure(figsize=(20, 2))
sns.countplot(y="Sex", palette="pastel", data=train_data);

In [ ]:
sns.catplot(x="Sex", y="Survived", hue="Pclass", palette="pastel", kind="bar", data=train_data)

In [ ]:
train_data.head()

## 📌 Tarefa 5:  Pré-processamento III
    Codificando Dados Categóricos
---

### Codificando Features

Documentação Pandas get dummies: [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

**Conjunto de dados de Treino**

In [ ]:
# Encoding the categorical columns
embarked_oh = pd.get_dummies(train_data["Embarked"], prefix="embarked")
sex_oh = pd.get_dummies(train_data["Sex"], prefix="sex")
plcass_oh = pd.get_dummies(train_data["Pclass"], prefix="pclass")

# Combine the encoded columns
train_data = pd.concat([train_data, 
                    embarked_oh, 
                    sex_oh, 
                    plcass_oh], axis=1)

train_data.head()

In [ ]:
# Drop the original categorical columns (because now they've been encoded)
train_data = train_data.drop(["Pclass", "Sex", "Embarked"], axis=1)

train_data.head()

---
**Conjunto de Dados de Teste**

In [ ]:
# Encoding the categorical columns
test_embarked_oh = pd.get_dummies(test_data["Embarked"], prefix="embarked")
test_sex_oh = pd.get_dummies(test_data["Sex"], prefix="sex")
test_plcass_oh = pd.get_dummies(test_data["Pclass"], prefix="pclass")

# Combine the encoded columns
test_data = pd.concat([test_data, 
                    test_embarked_oh, 
                    test_sex_oh, 
                    test_plcass_oh], axis=1)

test_data.head()

In [ ]:
# Drop the original categorical columns (because now they've been one hot encoded)
test_data = test_data.drop(["Pclass", "Sex", "Embarked"], axis=1)

test_data.head()

## 📌 Tarefa 6: Dividindo os conjuntos de dados em treinamento e teste
---

Dividir o conjunto de dados em treinamento e teste é muito comum, e você fará isso em inúmeras ocasiões. Mesmo que neste problema atual, tenhamos nosso treinamento e teste em arquivos csvs separadamente, usaremos essa técnica em nosso conjunto de dados de treinamento, para que você possa se acostumar com esse procedimento.

**train_test_split**: O primeiro argumento será o `feature data`, o segundo o` target or labels`. O argumento `test_size` especifica qual proporção dos dados originais é usada para o conjunto de teste. Por último, o kward `random_state` define uma semente para o gerador de números aleatórios que divide os dados em treino e teste.

Dividindo os dados de treinamento, usaremos parte de nossos dados de treinamento (30% neste caso) para testar a precisão de nossos diferentes modelos.

In [ ]:
from sklearn.model_selection import train_test_split

y = train_data["Survived"] #target
X = train_data.drop(["Survived", "PassengerId"], axis=1) #train predictors

#train_test_split(predictors, target, test_size = 0.22, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state= 21, stratify=y)

In [ ]:
X.shape

In [ ]:
# our target is a unique vector with one coordinate
y.shape

## 📌 Tarefa 7: Construindo nossos modelos de aprendizado de máquina
---


**Logistic Regression**

A regressão logística é uma técnica estatística que tem como objetivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias. Trata-se de um modelo de regressão para variáveis dependentes ou de resposta binomialmente distribuídas. É útil para modelar a probabilidade de um evento ocorrer como função de outros factores, como o problema do Titanic desse projeto. É um modelo linear generalizado que usa como função de ligação a função logit.

Reference: [Wikipedia](https://en.wikipedia.org/wiki/Logistic_regression)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
acc_logreg

**Decision Tree**

O aprendizado da árvore de decisão é uma das abordagens de modelagem preditiva usadas em estatística, mineração de dados e aprendizado de máquina. Ele usa uma árvore de decisão (como um modelo preditivo) para ir de observações sobre um item (representado nos ramos) para conclusões sobre o valor alvo - target - do item (representado nas folhas). Os modelos de árvore em que a variável de destino pode assumir um conjunto discreto de valores são chamados de árvores de classificação; nessas estruturas de árvore, as folhas representam rótulos de classe e os ramos representam conjunções de features que levam a esses rótulos de classe. As árvores de decisão nas quais a variável de destino pode assumir valores contínuos (normalmente números reais) são chamadas de árvores de regressão. As árvores de decisão estão entre os algoritmos de aprendizado de máquina mais populares devido à sua inteligibilidade e simplicidade. 

Reference: [Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
acc_dt = round(accuracy_score(y_pred, y_val) * 100, 2)
acc_dt

### Modelos Adicionais
---

Se você quiser testar outros modelos e comparar seu desempenho com os dois que já usamos. Abaixo você encontra esses modelos já declarados. 
Sinta-se à vontade para testá-los também, execute para ver os resultados.

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
acc_rf = round(accuracy_score(y_pred, y_val) * 100, 2)
print("Random Forest Accuracy",acc_rf)


#KNN | K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print("K-Nearest Neighbors Accuracy",acc_knn)


# Support Vector Machine
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print("Support Vector Machine", acc_svc)

## 📌 Tarefa 8: Realizando a submissão do projeto no Kaggle
---

Agora que você avaliou a perfomance do seu modelo, vamos criar um arquivo csv para a submissão na competição do Kaggle!

In [ ]:
#Original test dataset
test_data.head()

In [ ]:
test_data.shape

In [ ]:
pd.isnull(test_data).sum()

In [ ]:
test_data = test_data.fillna(0)
pd.isnull(test_data).sum()

Predição Final e Geração do Arquivo de Submissão no Kaggle

In [ ]:
submission = pd.DataFrame()
#set the output as a dataframe and convert to csv file named submission.csv
submission["PassengerId"] = test_data["PassengerId"]
submission

In [ ]:
test_data = test_data.drop("PassengerId", axis=1)
test_data.head()

In [ ]:
predictions = logreg.predict(test_data)
submission["Survived"] =  predictions
submission.to_csv("submission.csv", index=False)

### Dicas e Materiais de Estudo Recomendados: 

- [Seaborn: Statistical Data Visualization](https://seaborn.pydata.org/index.html)

- Recomendação que pode ser interessante, caso queira ver uma análise diferente dos mesmos dados do Titanic: [Titanic - A Data Science Approach](https://www.kaggle.com/pedrodematos/titanic-a-complete-data-science-approach)

- O notebook completo desse curso, você encontra na conta do github da instrutora. Se você desehar realizar melhorias, sinta-se a vontade para compartilhar com a instrutora. 
[Project on Github](https://github.com/mirianfsilva/titanic-kaggle-competition)

- Continue praticando no Kaggle! 

- Esse é um projeto prático, mas é importante que você também melhore seu background teorico em aprendizagem de máquina, entendendo como cada modelo funciona por trás do código. Então recomento esse curso do Coursera da Universidade de Stanford, que definitivamente é um excelente começo para quem é iniciante, e é um bom complemento para suas práticas: [Machine Learning by Stanford University](https://www.coursera.org/learn/machine-learning)